### <center> Calculating GSD from UAV image EXIF data using [PyExifTool](https://pypi.org/project/PyExifTool/) </center>
##### Requires Phil Harvey's [ExifTool](https://exiftool.org/) be installed and on system PATH


In [7]:
import os
import subprocess
import exiftool
import pandas as pd

# Check if ExifTool is accessible
try:
    subprocess.run(["exiftool", "-ver"], check=True)
    print("ExifTool is accessible.")
except subprocess.CalledProcessError:
    print("ExifTool is not accessible.")
print()

def GSD_from_exif_batch(folder):
    """
    Calculate ground sampling distance (GSD) in cm/pixel from Exif data for all images in folder
    :param folder: path to folder containing images
    :return: csv file containing Exif data and GSD for all images in folder
    """
    # Get list of .JPG,.PNG, or .TIF files in folder
    files = [file for file in os.listdir(folder) if file.endswith((".JPG", ".PNG", ".TIF"))]

    # Add folder path to each file and store as string in quotations
    files = [os.path.join(folder, file) for file in files]

    # Print all tags for first file in folder
    with exiftool.ExifToolHelper() as et:
        for d in et.get_metadata(files[0]):
            for k, v in d.items():
                print(f"{k} = {v}")
    print()

    # Empty list for storing Exif data
    Exif_data = []

    # Values for calculating ground sampling distance (GSD) W,H in cm/pixel are: SensorWidth (mm),
    # SensorHeight (mm), RelativeAltitude (m), FocalLength (mm), ImageWidth (px), ImageHeight (px)

    # Store specified tags as list of dictionaries for all files in folder
    with exiftool.ExifToolHelper() as et:
        for d in et.get_tags(files[0:-1], tags=["SourceFile", "FileName", "ImageWidth", "ImageHeight", # specify tags here
                                                "FocalLength", "RelativeAltitude"]):
            # append d to Exif_data list
            Exif_data.append(d)

    # Exif_data is a list of dictionaries. Convert to pandas dataframe where each unique value for SourceFile is a row
    df = pd.DataFrame(Exif_data)

    # Clean up column names; remove the text before the colon
    df.columns = df.columns.str.split(":").str[-1]

    # remove plus sign from RelativeAltitude
    df["RelativeAltitude"] = df["RelativeAltitude"].str.replace("+", "")

    # Convert columns to float
    df["RelativeAltitude"] = df["RelativeAltitude"].astype(float)
    df["FocalLength"] = df["FocalLength"].astype(float)
    df["ImageWidth"] = df["ImageWidth"].astype(float)
    df["ImageHeight"] = df["ImageHeight"].astype(float)

    # Add sensor width to dataframe (UAV is DJI Mavic 3M)
    # https://sdk-forum.dji.net/hc/en-us/articles/12325496609689-What-is-the-custom-camera-parameters-for-Mavic-3-Enterprise-series-and-Mavic-3M-
    df["SensorWidth"] = 17.4

    # Add GSDw and to dataframe
    # GSDw = (SensorWidth * RelativeHeight * 100) / (FocalLength * ImageWidth)
    df["GSDw"] = (df.SensorWidth * df.RelativeAltitude * 100) / (df.FocalLength * df.ImageWidth)

    # Save dataframe to csv
    df.to_csv(os.path.join(folder, f"Exif_data_{df['GSDw'].mean():.2f}.csv"), index=False) # save mean GSDw in filename

ExifTool is accessible.



In [9]:
# Define paths to folders containing images
folders = ["S:/Zack/Imagery/Chestnut/Annotation/Ohio/Route9/20230823_Route9-Orchard1perp_0.39GSD/",
           "S:/Zack/Imagery/Chestnut/Annotation/Ohio/Route9/20230824_Route9-Orchard1perp_0.84GSD/",
           "S:/Zack/Imagery/Chestnut/Annotation/Ohio/Route9/20230824_Route9-Orchard2_0.86GSD/",
           "S:/Zack/Imagery/Chestnut/Annotation/Ohio/Route9/20230824_Route9-Orchard3-closePar_RGB_0.67GSD/",
           "S:/Zack/Imagery/Chestnut/Annotation/Ohio/Route9/DJI_202308231046_004_Route9Orchard1South_0.75GSD/",
           "S:/Zack/Imagery/Chestnut/Annotation/Ohio/Route9/DJI_202308231102_005_Route9Orchard1North_1.60GSD/"]

# Call function for each folder in folders
for folder in folders:
    GSD_from_exif_batch(folder) # saves csv file with GSD in folder


SourceFile = S:/Zack/Imagery/Chestnut/Annotation/Ohio/Route9/20230823_Route9-Orchard1perp_0.39GSD/DJI_20230823095837_0001_D.JPG
ExifTool:ExifToolVersion = 12.7
File:FileName = DJI_20230823095837_0001_D.JPG
File:Directory = S:/Zack/Imagery/Chestnut/Annotation/Ohio/Route9/20230823_Route9-Orchard1perp_0.39GSD
File:FileSize = 9195520
File:FileModifyDate = 2023:08:23 09:58:40-05:00
File:FileAccessDate = 2023:12:08 10:02:10-06:00
File:FileCreateDate = 2023:12:04 14:58:10-06:00
File:FilePermissions = 100666
File:FileType = JPEG
File:FileTypeExtension = JPG
File:MIMEType = image/jpeg
File:ExifByteOrder = II
File:ImageWidth = 5280
File:ImageHeight = 3956
File:EncodingProcess = 0
File:BitsPerSample = 8
File:ColorComponents = 3
File:YCbCrSubSampling = 2 2
EXIF:ImageDescription = default
EXIF:Make = DJI
EXIF:Model = M3M
EXIF:Orientation = 1
EXIF:XResolution = 72
EXIF:YResolution = 72
EXIF:ResolutionUnit = 2
EXIF:Software = 10.12.05.45
EXIF:ModifyDate = 2023:08:23 09:58:37
EXIF:YCbCrPositioning = 2